In [ ]:
!pip install accelerate

In [ ]:
import os
import gc
import sys
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from accelerate import Accelerator
from transformers import (AutoModel,AutoTokenizer,AdamW,AutoModelForSequenceClassification,
                          AutoConfig,get_cosine_schedule_with_warmup)

from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [ ]:
group_map = {
    "LF":0,
    "MF":1,
    "UF":2,
    "competitor":3,
    "brand":4}

map_group = {
    0:"LF",
    1:"MF",
    2:"UF",
    3:"competitor",
    4:"brand"}

csv_path = '../input/google-ads-keyword-grouping2/fold{}.csv'
train_data = pd.DataFrame()
for i in range(5):
    df = pd.read_csv(csv_path.format(i))
    train_data = train_data.append(df)

train_data['target'] = train_data['Grouping'].map(group_map)
train_data.head()

In [ ]:
config = {
    'lr': 1.1e-5,
    'wd':1e-2,
    'batch_size':8,
    'valid_step':10,
    'num_workers':4,
    'max_len':10,
    'epochs':10,
    'nfolds':5,
    'seed':1000,
    'model_path':'roberta-large'
}

for i in range(config['nfolds']):
    os.makedirs(f'model{i}',exist_ok=True)

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

## Making Predictions

In [ ]:
test_data = pd.read_csv('../input/r-ur-roberta-train/test_data.csv')
test_data['Keyword'] = test_data['keyword1']

In [ ]:
class GadsTestDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.keywords = df['Keyword'].to_numpy()
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.keywords[idx],return_tensors='pt',
                                max_length=config['max_len'],
                                padding='max_length',truncation=True)
        return encode
    
    def __len__(self):
        return len(self.keywords)

In [ ]:
def get_prediction(df,path,model_path,device='cuda'):        
    model = AutoModelForSequenceClassification.from_pretrained(config['model_path'],num_labels = 5)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    predictions = list()
    
    for f in range(config['nfolds']):
        model.load_state_dict(torch.load(path.format(f),map_location=device))
        model.to(device)
        model.eval()

        test_ds = GadsTestDataset(df,tokenizer)
        test_dl = DataLoader(test_ds,
                            batch_size = config["batch_size"],
                            shuffle=False,
                            drop_last=False,
                            num_workers = 4,
                            pin_memory=True)

        with torch.no_grad():
            pred = list()
            for i, (inputs) in tqdm(enumerate(test_dl)):
                inputs = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs.items()}
                outputs = model(**inputs)
                outputs = outputs['logits'].cpu().detach().numpy().tolist()
                pred.extend(outputs)
            predictions.append(pred)
            
    torch.cuda.empty_cache()
    final_predictions = np.mean(predictions,axis=0)
    return np.mean(predictions,axis=0)

In [ ]:
model_paths = [
    ['../input/d/maunish/google-ads-keyword-grouping/model{0}/model{0}.bin',config['model_path']],
]

In [ ]:
predictions = np.zeros((test_data.shape[0],5))
for path,model_path in model_paths:
    predictions += get_prediction(test_data,path,model_path)/len(model_paths)

In [ ]:
predictions = np.argmax(predictions,axis=1)

In [ ]:
test_data['target'] = predictions
test_data['Grouping'] = test_data['target'].map(map_group)

In [ ]:
test_data.head()

In [ ]:
test_data.to_csv('test_df.csv',index=False)

In [ ]:
test_data['Grouping'].value_counts()